# Métodos de iteración funcional para sistemas de ecuaciones

Entradas:

 - una función $g: \mathbb{R}^N \to \mathbb{R}^N, g\in \mathscr{C}^1(\mathbb{R}^N)$,
 - $x_0 \in \mathbb{R}^N$.
 
y, según la variante:
 
  1. tolerancia entre dos iteraciones consecutivas,
  2. solución y tolerancia,
  3. constante de Lipstchiz y tolerancia.

Primero, definimos un generador de los términos de la sucesión para cada método.

In [68]:
def functional_iteration(g, x_0):
    x = vector(x_0)
    while True:
        yield x
        x = g(*x)

En el método de Newton, resolvemos el SEL correspondiente en lugar de hallar la inversa de la matriz.

In [69]:
def newton(g, x_0):
    x = vector(x_0)
    f = g.derivative()
    while True:
        yield x
        y = f(*x).solve_right(g(*x))
        x = vector(x) - vector(y)

Para la primera variante, creamos un generador que, a partir de uno de los métodos, nos dé la distancia entre dos aproximaciones consecutivas, y nos quedamos con el primer elemento del generador para el que la distancia con la aproximación anterior sea menor que la tolerancia dada.

In [78]:
def adjacent_distance(method, g, x_0):
    gen = method(g, x_0)
    x = gen.next()
    
    for i in gen:
        yield (i,abs(x-i))
        x = i
        

In [73]:
def first_variant(method, g, x_0, epsilon):
    orig_gen = adjacent_distance(method,g,x_0)
    gen = it.takewhile(lambda (x,e) : e >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

Para la segunda variante, procedemos de forma similar y creamos un generador que pare en la primera aproximación que nos proporcione un error menor que la tolerancia.

In [74]:
def second_variant(method, g, x_0, sol, epsilon):
    orig_gen = method(g,x_0)
    gen = it.takewhile(lambda (x) : abs(x-sol) >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

En este caso, tenemos $L\in (L_0,1)$, con $L_0$ la constante de Lipschitz de $g$, y el teorema del punto fijo nos da la cota

$$|x_n-x^*| \le \frac{L^n}{1-L}|g(x_0)-x_0|$$

Ahora, si imponemos 

$$\frac{L^n}{1-L}|g(x_0)-x_0| < \varepsilon$$

tenemos que

$$ n > \frac{\log\left(\displaystyle\frac{\varepsilon(1-L)}{|g(x_0)-x_0|}\right)}{\log(L)} $$

In [79]:
def third_variant(g, x_0, L, epsilon):
    n_iterations = int((log(epsilon*(1-L)/abs(vector(g(*x_0))-vector(x_0)))/log(L)))+2
    
    print("Iterations: {}".format(n_iterations))
    
    return it.islice(functional_iteration(g,x_0), n_iterations)

In [76]:
def print_all(gen):
    for i in gen:
        print(i)
        
def print_last(gen):
    j = None
    for j in gen:
        pass
    print(j)

In [77]:
contractive2(x,y) = (0.5*x+3, 0.5*y)

print("Todos las aproximaciones de la primera variante")
gen = first_variant(newton, contractive2, (0.5,.5), 0.0000005)

fix_point, error = None, None
for fix_point, error in gen:
    print("Aproximación: {}, distancia entre términos: {}".format(fix_point, error))
    
print("Todos las aproximaciones de la tercera variante")
gen = third_variant(contractive2, (5,5), 0.8, 10e-9)

fix_point = None, None
for fix_point in gen:
    print("Aproximación: {}, error: {}".format(fix_point, abs(vector((6,0))-fix_point)))

print("")
print("Última aproximación de la segunda variante")
print_last(second_variant(functional_iteration, contractive2, (5,5), fix_point, 10e-9))

Todos las aproximaciones de la primera variante
Aproximación: (-6.00000000000000, 0.000000000000000), distancia entre términos: 6.51920240520265
Aproximación: (-6.00000000000000, 0.000000000000000), distancia entre términos: 0
Todos las aproximaciones de la tercera variante
Iterations: 95
Aproximación: (5, 5), error: sqrt(26)
Aproximación: (5.50000000000000, 2.50000000000000), error: 2.54950975679639
Aproximación: (5.75000000000000, 1.25000000000000), error: 1.27475487839820
Aproximación: (5.87500000000000, 0.625000000000000), error: 0.637377439199098
Aproximación: (5.93750000000000, 0.312500000000000), error: 0.318688719599549
Aproximación: (5.96875000000000, 0.156250000000000), error: 0.159344359799775
Aproximación: (5.98437500000000, 0.0781250000000000), error: 0.0796721798998873
Aproximación: (5.99218750000000, 0.0390625000000000), error: 0.0398360899499436
Aproximación: (5.99609375000000, 0.0195312500000000), error: 0.0199180449749718
Aproximación: (5.99804687500000, 0.00976562500